In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import cv2
import keras
from keras.models import Sequential,Model
from keras.layers import Dense,MaxPooling2D,Add,ReLU,Activation,Conv2D,Flatten,Reshape,PReLU,Concatenate,Conv2DTranspose,Dropout,BatchNormalization,Lambda,Input
from keras.callbacks import ModelCheckpoint
from keras.layers.advanced_activations import LeakyReLU 
from keras.layers import Concatenate
from keras import optimizers
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [ ]:
def down_sample_2D(inputs,channels,out_size=(6,14)):
    x=Conv2D(channels,kernel_size=1,strides=(1,1))(inputs)
    x=tf.image.resize(x,out_size,method='bilinear',preserve_aspect_ratio=False,antialias=True,name=None)
    return x

In [ ]:
def face_detector():
    ip=Input(shape=(64,128,3))
    x=Lambda(lambda x : (x/255))(ip)
    
    x=Conv2D(64,kernel_size = 3,strides = (1,1), padding='same')(x)
    x=Conv2D(64,kernel_size = 3,strides = (1,1), padding='valid')(x)
    
    x=Conv2D(64,kernel_size = 3,strides = (2,2), padding='same')(x)
    x=ReLU()(x)
    x=BatchNormalization()(x)
    
    x=Conv2D(128,kernel_size = 3,strides = (1,1), padding='same')(x)
    x=Conv2D(128,kernel_size = 3,strides = (1,1), padding='same')(x)
    
    x=Conv2D(128,kernel_size = 3,strides = (2,2), padding='valid')(x)
    x=ReLU()(x)
    x=BatchNormalization()(x)
    
    x=Conv2D(256,kernel_size = 3,strides = (1,1), padding='same')(x)
    x=Conv2D(256,kernel_size = 3,strides = (1,1), padding='same')(x)
    x=Conv2D(256,kernel_size = 3,strides = (1,1), padding='same')(x)
    x=Conv2D(256,kernel_size = 3,strides = (1,1), padding='same')(x)
    
    x=Conv2D(256,kernel_size = 3,strides = (2,2), padding='valid')(x)
    x=ReLU()(x)
    x=BatchNormalization()(x)
    
    x=Conv2D(512,kernel_size = 3,strides = (1,1), padding='same')(x)
    x=Conv2D(512,kernel_size = 3,strides = (1,1), padding='same')(x)
    x=Conv2D(512,kernel_size = 3,strides = (1,1), padding='same')(x)
    x=Conv2D(512,kernel_size = 3,strides = (1,1), padding='same')(x)
    
    x=Conv2D(256,kernel_size = 3,strides = (2,2), padding='valid')(x)
    x=ReLU()(x)
    x=BatchNormalization()(x)
    
    x=Conv2D(128,kernel_size = 3,strides = (1,1), padding='same')(x)
    x=Conv2D(128,kernel_size = 3,strides = (1,1), padding='same')(x)
    x=Conv2D(128,kernel_size = 3,strides = (1,1), padding='same')(x)
    
    x=Flatten()(x)
    x=Dense(388)(x)
    
    model=Model(inputs=ip,outputs=x)
    return model
fd=face_detector()
fd.summary()
    

In [ ]:
def face_detector1():
    ip=Input(shape=(64,128,3))
    x=Lambda(lambda x : (x/255))(ip)
    x1=Conv2D(64,kernel_size = 3,strides = (1,1), padding='same')(x)
    x=ReLU()(x1)
    x=Conv2D(64,kernel_size = 3,strides = (1,1), padding='same')(x)
    x=ReLU()(x)
    x=Conv2D(64,kernel_size = 3,strides = (1,1), padding='same')(x)
    x=ReLU()(x)
    x=BatchNormalization()(x)
    x=Add()([x1,x])
    x2=Conv2D(128,kernel_size = 3,strides = (1,1), padding='same')(x)
    x=ReLU()(x)
    x=Conv2D(128,kernel_size = 3,strides = (1,1), padding='same')(x)
    x=ReLU()(x)
    x=Conv2D(128,kernel_size = 3,strides = (1,1), padding='same')(x)
    x=ReLU()(x)
    x=BatchNormalization()(x)
    x=Add()([x2,x])
    x=Conv2D(256,kernel_size = 3,strides = (2,2), padding='valid')(x)
    x=Conv2D(256,kernel_size = 3,strides = (2,2), padding='valid')(x)
    x=Conv2D(256,kernel_size = 3,strides = (2,2), padding='valid')(x)
    x=Conv2D(256,kernel_size = 3,strides = (2,2), padding='valid')(x)
    x=ReLU()(x)
    x=BatchNormalization()(x)
    x=Conv2D(256,kernel_size = 3,strides = (1,1), padding='same')(x)
    x=Conv2D(256,kernel_size = 3,strides = (1,1), padding='same')(x)
    x=Flatten()(x)
    x=Dense(388)(x)
    
    model=Model(inputs=ip,outputs=x)
    return model
    
fd=face_detector1()
fd.summary()

In [ ]:
x_full=[]
x_fd_train=[]
y_fd_train=[]
data=np.load('../input/helen-data/helen_data.npy',allow_pickle=True)
for i in range(len(data)):
    img=data[i][0][:64,:,:]
    x_full.append(data[i][0])
    x_fd_train.append(img)
    y_fd_train.append(data[i][1]+data[i][2])
    
sc=StandardScaler()
y_fd_train1=sc.fit_transform(y_fd_train)

In [ ]:
x_fd_train=np.array(x_fd_train)
y_fd_train1=np.array(y_fd_train1)
opt=keras.optimizers.Adam(lr=0.0009)
fd.compile(optimizer=opt, loss='mean_squared_error',metrics=['accuracy'])
history=fd.fit(x_fd_train,y_fd_train1,batch_size=16,epochs=30,verbose=1,validation_split=0.01)

In [ ]:
ind=2210
tst_im=x_full[ind]
tst_pts=y_fd_train[ind]
im=np.resize(x_fd_train[ind],(1,64,128,3))
lower=np.zeros((128,128,3),dtype=np.int8)
pred_pts=fd.predict(im)
pts=sc.inverse_transform(pred_pts)
pts=pts[0]
x_t=list(map(int,tst_pts[:194]))
y_t=list(map(int,tst_pts[194:]))
x=list(map(int,pts[:194]))
y=list(map(int,pts[194:]))
#img=x_train[250]
for i in range(194):
    #cv2.circle(tst_im,(x_t[i],y_t[i]),1,(255,0,0),1)
    cv2.circle(lower,(x[i],y[i]),1,(255,0,0),1)
    
img=np.concatenate((tst_im[:64],lower[64:]),axis=0)
plt.subplot(1,2,1)
plt.imshow(img)
plt.subplot(1,2,2)
plt.imshow(tst_im)

In [ ]:
cv2.imwrite('tr4.jpg',img)
cv2.imwrite('ts4.jpg',tst_im)

In [ ]:
from keras.utils import plot_model
plot_model(fd, to_file='fd_model.png')

In [ ]:
def encoder():
    ip=Input(shape=(64,128,3))
    #org=Input(shape=(128,128,3))
    x1=Lambda(lambda x : (x/255))(ip)
    #Encoder
    x=Conv2D(8,kernel_size = 3,strides = (1,1))(x1)
    x=BatchNormalization()(x)
    x=PReLU()(x)
    
    x=Conv2D(8,kernel_size = 3,strides = (1,1))(x)
    x=BatchNormalization()(x)
    x=PReLU()(x)
    #model.add(BatchNormalization())
    x=Conv2D(16,kernel_size = 3,strides = (1,1))(x)
    x=BatchNormalization()(x)
    x=PReLU()(x)
    
    #d=Lambda(lambda i : down_sample_2D(i,32,out_size=(6,14)))(x)
    d=Conv2D(64,kernel_size=(11,11), strides=(8,8))(x)
    #model.add(BatchNormalization())
    x=Conv2D(16,kernel_size = 3,strides = (2,2))(x)
    x=BatchNormalization()(x)
    x=PReLU()(x)
    #model.add(BatchNormalization())        
    x=Conv2D(32,kernel_size = 3,strides = (2,2))(x)
    x=BatchNormalization()(x)
    x=PReLU()(x)
    #model.add(BatchNormalization())
    x=Conv2D(64,kernel_size = 3,strides = (2,2))(x)
    x=BatchNormalization()(x)
    x=PReLU()(x)
    #fully connected
    x=Add()([x,d])
    x=Flatten()(x)
    x=Dense(3708)(x)
    
    model=Model(inputs=ip,outputs=x)
    return model

    #Decoder
def decoder():
    ip=Input(shape=(None,3708+388))
    x=Reshape(target_shape=(4,8,128),input_shape = (3708+388,))(ip)
    x=Conv2DTranspose(64,kernel_size = 9,strides =(1,1),padding = "SAME")(x)
    x=BatchNormalization()(x)
    x=LeakyReLU(0.2)(x)
    #model.add(BatchNormalization())
    x=Conv2DTranspose(32,kernel_size = 6,strides =(2,2),padding = "SAME")(x)
    x=BatchNormalization()(x)
    x=LeakyReLU(0.2)(x)
    #model.add(BatchNormalization())
    x=Conv2DTranspose(16,kernel_size = 3,strides =(2,2),padding = "SAME")(x)
    x=BatchNormalization()(x)
    x=LeakyReLU(0.2)(x)
    #model.add(BatchNormalization())
    x=Conv2DTranspose(8,kernel_size = 3,strides =(2,2),padding = "SAME")(x)
    x=BatchNormalization()(x)
    x=LeakyReLU(0.2)(x)
    #model.add(BatchNormalization())
    x=Conv2DTranspose(3,kernel_size = 2,strides =(2,2),padding = "SAME")(x)
    x=BatchNormalization()(x)
    x=LeakyReLU(0.2)(x)
    
    #out=Concatenate(axis=1)([ip,x])
    
    model=Model(inputs=ip,outputs=x)
    return model

In [ ]:
e=encoder()
e.summary()

In [ ]:
def sr():
  ip=Input(shape=(64,128,3))
  #x=Lambda(lambda x:x/255)(ip)

  x=Conv2D(64,kernel_size=9,strides=(1,1),padding='same')(ip)
  x1=PReLU()(x)

  x=Conv2D(64,kernel_size=3,strides=(1,1),padding='same')(x1)
  x=BatchNormalization()(x)
  x=PReLU()(x)
  x=Conv2D(64,kernel_size=3,strides=(1,1),padding='same')(x)
  x=BatchNormalization()(x)
  x2=Add()([x1,x])

  x=Conv2D(64,kernel_size=3,strides=(1,1),padding='same')(x2)
  x=BatchNormalization()(x)
  x=PReLU()(x)
  x=Conv2D(64,kernel_size=3,strides=(1,1),padding='same')(x)
  x=BatchNormalization()(x)
  x3=Add()([x2,x])

  x=Conv2D(64,kernel_size=3,strides=(1,1),padding='same')(x3)
  x=BatchNormalization()(x)
  x=PReLU()(x)
  x=Conv2D(64,kernel_size=3,strides=(1,1),padding='same')(x)
  x=BatchNormalization()(x)
  x4=Add()([x3,x])

  x=Conv2D(64,kernel_size=3,strides=(1,1),padding='same')(x4)
  x=BatchNormalization()(x)
  x=PReLU()(x)
  x=Conv2D(64,kernel_size=3,strides=(1,1),padding='same')(x)
  x=BatchNormalization()(x)
  x5=Add()([x4,x])

  x=Conv2D(64,kernel_size=3,strides=(1,1),padding='same')(x5)
  x=BatchNormalization()(x)
  x=PReLU()(x)
  x=Conv2D(64,kernel_size=3,strides=(1,1),padding='same')(x)
  x=BatchNormalization()(x)
  x6=Add()([x5,x])

  x=Conv2D(64,kernel_size=3,strides=1,padding='same')(x6)
  x=BatchNormalization()(x)
  x7=Add()([x1,x])

  x=Conv2D(256,kernel_size=3,strides=1,padding='same')(x7)
  #x=UpSampling2D(size=2)(x)
  x=PReLU()(x)

  x=Conv2D(256,kernel_size=3,strides=1,padding='same')(x)
  #x=UpSampling2D(size=2)(x)
  x=PReLU()(x)

  x=Conv2D(3,kernel_size=9,strides=1,padding='same')(x)

  model=Model(inputs=ip,outputs=x)
  return model

In [ ]:
#s=sr()
opt=keras.optimizers.Adam(lr=0.0006)
s.compile(optimizer=opt, loss=norm_loss,metrics=['accuracy'])
history=s.fit(x_sr,y_sr,batch_size=16,epochs=10,verbose=1,validation_split=0.002)

In [ ]:
for layer in s.layers[:39]:
    layer.trainable=False
s.summary()

In [ ]:
for layer in fd.layers:
    layer.trainable=False
fd.summary()

In [ ]:
ip=Input(shape=(64,128,3))
f=fd(ip)
e=encoder()(ip)
dense_rep=keras.layers.concatenate([f,e],axis=1)
d=decoder()(dense_rep)
sr=s(d)
ae=Model(inputs=ip,outputs=sr)
#ae=Sequential()
#ae.add(f)
#ae.add(e)
#ae.add(d)
#ae.add(s)
ae.summary()

In [ ]:
import tensorflow as tf
def norm_loss(a,b):
    loss=tf.math.reduce_mean((((a/255))-b)**2)
    return loss

In [ ]:
opt=keras.optimizers.Adam(lr=0.0008)
ae.compile(optimizer=opt, loss=norm_loss,metrics=['accuracy'])

In [ ]:
history=ae.fit(x_train,y_train,batch_size=16,epochs=10,verbose=1,validation_split=0.01)

In [ ]:
ae.save('AESRFD_0.0044_7K.h5')

In [ ]:
srgan=keras.models.load_model('../input/superresolution/SrGanRGB_3e-4.h5',custom_objects={'norm_loss':norm_loss})
srgan.summary()

In [ ]:
model=Model(inputs=srgan.inputs,outputs=srgan.layers[21].output)
model.summary()

In [ ]:
out=ae.predict(np.resize(x_train[178],(1,64,128,3)))
out=np.resize(out,(64,128,3))/10
#out=np.multiply(out,255)
up=x_train[178]
plt.subplot(2,2,1)
plt.imshow(up)
plt.subplot(2,2,2)
plt.imshow(up)
plt.subplot(2,2,3)
plt.imshow(out)
plt.subplot(2,2,4)
plt.imshow(y_train[178])

In [ ]:
np.average(out)

In [ ]:
import matplotlib.pyplot as plt
'''square = 2
ix = 1
for _ in range(square):
	for _ in range(square):
		# specify subplot and turn of axis
		ax = plt.subplot(square, square, ix)
		ax.set_xticks([])
		ax.set_yticks([])
		# plot filter channel in grayscale
		plt.imshow(f[0, :, :, ix-1], cmap='gray')
		ix += 1
# show the figure
plt.show()'''
plt.imshow(f[0,:,:,23],cmap='gray')

In [ ]:
for layer in s.layers[:39]:
    layer.trainable=False
#len(srgan.layers)
s.summary()

In [ ]:
f_model=Sequential()
f_model.add(g)
f_model.add(srgan)
f_model.summary()

In [ ]:
x_train=[]
y_train=[]
n=0

In [ ]:
cascade=cv2.CascadeClassifier('../input/haarcascadefrontalfaces/haarcascade_frontalface_default.xml')
for loc in os.listdir('../input/celeba-dataset/img_align_celeba/img_align_celeba/'):
    im=cv2.imread('../input/celeba-dataset/img_align_celeba/img_align_celeba/'+loc)
    gray=cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    faces=cascade.detectMultiScale(gray,1.3,5)
    for (x,y,w,h) in faces:
        face=im[y:y+h,x:x+w]
    if n<7000:
        face=cv2.resize(face,(128,128))
        face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        #c=face[:64,:]
        #b=cv2.blur(face[64:,:],(10,10))
        #b=np.full((64,128,3),255)
        #x=np.concatenate((c,b),axis=0)
        #y=cv2.resize(face,(128,128))
        x_train.append(face[:64,:])
        y_train.append(face[64:,:])
        if n%1000==0:
            print(n,end=' ')
        n+=1
    else:
        break    

In [ ]:
k=0
x_sr=[]
y_sr=[]
cascade=cv2.CascadeClassifier('../input/haarcascadefrontalfaces/haarcascade_frontalface_default.xml')
for loc in os.listdir('../input/celeba-dataset/img_align_celeba/img_align_celeba/'):
    im=cv2.imread('../input/celeba-dataset/img_align_celeba/img_align_celeba/'+loc)
    gray=cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    faces=cascade.detectMultiScale(gray,1.3,5)
    for (x,y,w,h) in faces:
        face=im[y:y+h,x:x+w]
    if k<5000:
        face=cv2.resize(face,(128,128))
        face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        #c=face[:64,:]
        #b=cv2.blur(face[64:,:],(10,10))
        #b=np.full((64,128,3),255)
        #x=np.concatenate((c,b),axis=0)
        #y=cv2.resize(face,(128,128))
        x_sr.append(cv2.blur(face[64:,:],(10,10)))
        y_sr.append(face[64:,:])
        if k%500==0:
            print(k,end=' ')
        k+=1
    else:
        break    

In [ ]:
x_sr=np.array(x_sr)
x_sr=x_sr/255.0
y_sr=np.array(y_sr)

In [ ]:
opt=keras.optimizers.Adam(lr=0.001)
f_model.compile(optimizer=opt, loss=norm_loss,metrics=['accuracy'])

In [ ]:
x_train=np.array(x_train)
y_train=np.array(y_train)
y_train.shape
#plt.imshow(np.resize(x_train[0],(128,128,3)))

In [ ]:
history=f_model.fit(x_train,y_train,batch_size=16,epochs=10,verbose=1,validation_split=0.01)

In [ ]:
#import cv2
#import matplotlib.pyplot as plt
#%matplotlib inline
#img=cv2.imread('../input/images1/Capture1.PNG')
ip=x_train[23]
#ip=cv2.resize(img,(128,128))
ip=cv2.cvtColor(ip,cv2.COLOR_BGR2RGB)

In [ ]:
import numpy as np
#op=f_model.predict(ip)
plt.imshow(np.resize(x_train[1],(128,128,3)))

In [ ]:
a=(3,128,128,3)
b=a[0]//2
print(b)